In [102]:
import pandas as pd
import xarray as xr
import _pickle as cPickle

metadata = pd.read_csv('data/metadata.csv', index_col=0)

topic_model = cPickle.load(open('data/topic_model.pkl', 'rb'))
s_anchor = pd.read_csv('data/s_anchor.csv', index_col=0)['anchor words']

with open('data/feature_names.txt', 'r', encoding='utf-8') as f:
    feature_names = [feat.rstrip() for feat in f.readlines()]

In [45]:


doc_topic_prob = topic_model.p_y_given_x

n_topics = doc_topic_prob.shape[1]
n_docs = doc_topic_prob.shape[0]

topic_names = s_anchor.index.values
doc_names = metadata.index.values


da_doc_topic = xr.DataArray(doc_topic_prob, coords= {'topic': topic_names, 'doc' : doc_names}, dims = ['doc', 'topic'])

In [68]:
def get_aligned_docs(da_doc_topic, prob, num=10):
    """Returns docs with max probability for given topic sorted by MA probability. Seems many documents have the same 99.999% topic probability so downselecting with MA""" 

    aligned_docs = []
    for topic in da_doc_topic.coords['topic']:
        s = da_doc_topic.sel(topic=topic).to_series()
        docs = s.sort_values(ascending=False)
        docs = docs.where(docs == docs.iloc[0]).dropna()

        probs = prob[docs.index]
        probs = probs.sort_values(ascending=False)
        top_probs = probs[0:num]

        aligned_docs.append(list(top_probs.index.values))

    aligned_docs = pd.Series(aligned_docs, index = da_doc_topic.coords['topic'])

    return aligned_docs
aligned_docs = get_aligned_docs(da_doc_topic, metadata['prob'])
aligned_docs
# metadata['title'][docs.index]

topic
topic_0     [19355, 19359, 19360, 19363, 13461, 19367, 193...
topic_1     [14865, 14874, 14876, 14878, 14888, 14890, 148...
topic_2     [19360, 19363, 5, 24044, 17, 34, 7094, 19410, ...
topic_3     [19355, 19360, 19363, 13461, 19367, 19370, 193...
topic_4     [19355, 13461, 8, 19380, 19386, 19405, 19406, ...
topic_5     [24044, 22, 19386, 31, 7094, 24049, 58, 24051,...
topic_6     [24044, 9264, 9268, 24054, 9270, 24063, 9277, ...
topic_7     [24044, 22, 31, 58, 84, 24054, 9270, 7105, 240...
topic_8     [24044, 19410, 19436, 19441, 19447, 11748, 195...
topic_9     [24044, 21, 19390, 7094, 58, 7098, 19465, 7100...
topic_10    [19355, 19367, 21, 19390, 19405, 19406, 13469,...
topic_11      [9264, 39, 54, 68, 72, 93, 103, 116, 9276, 128]
topic_12                 [0, 3, 5, 8, 16, 37, 41, 48, 54, 70]
topic_13    [39, 82, 7100, 103, 110, 116, 131, 145, 208, 8...
topic_14        [22, 31, 34, 48, 54, 58, 114, 117, 7109, 153]
topic_15    [9264, 19453, 93, 9270, 7105, 103, 13488, 130,...
to

In [71]:
l = aligned_docs['topic_8']

metadata.loc[l]

,title,display_text,prob
ID,,,
24044,electrical energy storage systems a comparativ...,<a href=https://econpapers.repec.org/RePEc:ee...,5.667118e-08
19410,a review on electrochemical double layer capac...,<a href=https://www.infona.pl/resource/bwmeta...,2.446555e-08
19436,design and new control of dc dc converters to ...,<a href=https://dblp.uni-trier.de/db/journals...,1.957525e-08
19441,everlast long life supercapacitor operated wir...,<a href=http://ieeexplore.ieee.org/xpls/abs_a...,1.869518e-08
19447,zero current switching switched capacitor zero...,<a href=http://ira.lib.polyu.edu.hk/handle/10...,1.796213e-08
11748,flywheel batteries come around again,<a href=https://ieeexplore.ieee.org/document/...,1.451621e-08
19588,a power electronic interface for a battery sup...,<a href=http://yadda.icm.edu.pl/yadda/element...,1.084025e-08
24078,energy management in the decentralized generat...,<a href=https://hal.archives-ouvertes.fr/hal-...,1.074312e-08
9282,bulk energy storage potential in the usa curre...,<a href=https://www.sciencedirect.com/science...,1.067886e-08
